# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

We'll also simulate the timestamp of user interaction, with the following methodology:

1. We'll analyze the average delta in timestamps between consecutive ratings per user in the same dataset that the preference model was trained on (e.g: movielens) (present in data/movielens-1m/avg_str_time_dff_per_user.csv)
2. Assuming that the time_delta is normally distributed (with the mean and standart deviation being defined from the dataset), we'll randomly generate a delta_t for each user. the time_0 will be the timestamp when we start generating the user preferences.

In [1]:
from scipy.stats import expon
from tqdm import tqdm
import pandas as pd


In [2]:
import numpy as np

import torch

In [3]:
import os
os.getcwd()

'/home/caio/dev/dynamicTasteDistortion/simulation'

In [4]:
import sys
from pathlib import Path
import pickle
import os

In [5]:
sys.path.append('/home/caio/dev')

In [6]:
from dynamicTasteDistortion.simulationConstants import ML_1M_ORACLE_PATH

## Reading and setting up data

In [7]:
preference_matrix = pd.read_csv(f"../{ML_1M_ORACLE_PATH}")
preference_matrix = preference_matrix.rename(columns={"Rating": "rating"})



click_matrix = pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at", "timestamp"])

user_to_time_delta = pd.read_csv("../data/movielens-1m/median_time_diff_per_user.csv").set_index("userId")

In [8]:
preference_matrix

,Unnamed: 0,user,item,genres,rating
0,0,1,1193,drama,1
1,1,1,661,animation|children's|musical,0
2,2,1,914,musical|romance,0
3,3,1,3408,drama,1
4,4,1,2355,animation|children's|comedy,1
...,...,...,...,...,...
19574584,19574584,6040,2258,action,0
19574585,19574585,6040,2845,drama,0
19574586,19574586,6040,3607,comedy|drama|western,1
19574587,19574587,6040,690,romance,0


In [9]:
userToExpDistribution = {
    user: expon(scale=row["median_timestamp_diff"])
    for user, row in user_to_time_delta.iterrows()
}


unique_users = list(userToExpDistribution.keys())

unique_items = list(preference_matrix["item"].unique())



with open("userToExpDistribution.pkl", "wb") as f:
    pickle.dump(userToExpDistribution, f)

## Bootstrapping

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
'cuda' if torch.cuda.is_available() else 'cpu'

'cuda'

In [11]:
from dynamicTasteDistortion.simulation.simulator import Simulator
from dynamicTasteDistortion.simulation.simulationUtils import get_user_preferences, map_prediction_to_preferences, random_rec


In [12]:
oracle_tensor = get_user_preferences(preference_matrix)

In [13]:
oracle_tensor.shape

torch.Size([5289, 3701])

In [14]:
oracle_tensor.shape

torch.Size([5289, 3701])

In [15]:
oracle_tensor.shape

torch.Size([5289, 3701])

In [16]:
user_to_time_delta = pd.read_csv("../data/movielens-1m/median_time_diff_per_user.csv").set_index("userId")
user_to_exp_distribution = {
    user: expon(scale=row["median_timestamp_diff"])
    for user, row in user_to_time_delta.iterrows()
}


In [17]:
user_to_time_delta.keys()

Index(['median_timestamp_diff', 'std_timestamp_diff', 'n_entries'], dtype='object')

In [18]:
len(user_to_exp_distribution.keys())

5289

In [19]:
mapa = {user_id: idx for idx, user_id in enumerate(user_to_exp_distribution.keys())}

In [20]:
len(mapa.keys())

5289

In [21]:
len(mapa.values())

5289

In [22]:
sim = Simulator(oracle_matrix=preference_matrix, model=None, num_rounds=5, initial_date=0.0)

/home/caio/dev/dynamicTasteDistortion/simulation/simulator.py:104: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  boostrapped_df = pd.concat([boostrapped_df, round_df], ignore_index=True)


In [23]:
sim.users

tensor([   1,    2,    3,  ..., 6037, 6039, 6040], device='cuda:0')

In [24]:
sim.click_matrix

,user,item,feedback,clicked_at,timestamp
0,0,2911,1.0,0.0,0.141685
1,0,636,1.0,1.0,0.042141
2,0,1644,1.0,2.0,0.014910
3,0,1573,NaN,NaN,NaN
4,0,2748,NaN,NaN,NaN
...,...,...,...,...,...
528895,5288,1386,NaN,NaN,NaN
528896,5288,1675,0.0,NaN,NaN
528897,5288,2557,NaN,NaN,NaN
528898,5288,567,NaN,NaN,NaN


In [25]:
sim.bootstrap_clicks(k=100)

/home/caio/dev/dynamicTasteDistortion/simulation/simulator.py:104: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  boostrapped_df = pd.concat([boostrapped_df, round_df], ignore_index=True)


KeyboardInterrupt: 